In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
"""N = 2
M = 5
V = -1
dt = 0.01

time_options = np.linspace(0.1, 2.5, 50)
final_J_V_ratios = np.linspace(0, 1.5, 50)
time_options_grid, final_J_V_ratios_grid = np.meshgrid(time_options, final_J_V_ratios)
probability_grid = np.zeros_like(time_options_grid)

for idx1, time_option in enumerate(time_options):
    time_array = [time_option, time_option, time_option, time_option]

    for idx2, final_J_V_ratio in enumerate(final_J_V_ratios):

        J_V_ratios = [
            [0.1, 10],  # step 1
            [10, 10],   # step 2
            [10, 2],    # step 3
            [2, final_J_V_ratio]    # step 4
        ]

        mu_V_ratios = [
            [10, 10],  # step 1
            [10, 0],   # step 2
            [0, 0],    # step 3
            [0, 0]     # step 4
        ]

        times, J_V_ratio_routine, mu_V_ratio_routine = make_linear_stepped_routines(J_V_ratios, mu_V_ratios, time_array, dt)

        adiabatic_energies, adiabatic_wavefunctions, adiabatic_probabilities, adiabatic_overlaps, true_energies = \
            simulate_adiabatic_evolution(N, M, V, mu_V_ratio_routine, J_V_ratio_routine, times, dt=dt, initial_state=None)
            
        probability_grid[idx2, idx1] = adiabatic_probabilities[-1, 0]
"""

In [ ]:
"""plt.pcolormesh(time_options_grid, final_J_V_ratios_grid, probability_grid, shading = "auto", cmap = "plasma", vmin = 0, vmax = 1)
plt.colorbar(label="Ground State Probability")
plt.xlabel("$T/4$")
plt.ylabel("$J/|V|$")
plt.title("Lower Bound Probability Grid")
plt.xlim(0,1.6)"""

In [5]:
from concurrent.futures import ThreadPoolExecutor

N = 2
M = 5
V = -1
dt = 0.01

time_options = np.linspace(0.1, 2.5, 50)
final_J_V_ratios = np.linspace(0, 1.5, 50)
time_options_grid, final_J_V_ratios_grid = np.meshgrid(time_options, final_J_V_ratios)
probability_grid = np.zeros_like(time_options_grid)

def calculate_probability(time_option, final_J_V_ratio):
    time_array = [time_option, time_option, time_option, time_option]

    J_V_ratios = [
        [0.1, 10],  # step 1
        [10, 10],   # step 2
        [10, 2],    # step 3
        [2, final_J_V_ratio]  # step 4
    ]

    mu_V_ratios = [
        [10, 10],  # step 1
        [10, 0],   # step 2
        [0, 0],    # step 3
        [0, 0]     # step 4
    ]

    times, J_V_ratio_routine, mu_V_ratio_routine = make_linear_stepped_routines(J_V_ratios, mu_V_ratios, time_array, dt)

    adiabatic_energies, adiabatic_wavefunctions, adiabatic_probabilities, adiabatic_overlaps, true_energies = \
        simulate_adiabatic_evolution(N, M, V, mu_V_ratio_routine, J_V_ratio_routine, times, dt=dt, initial_state=None)
    
    probability = adiabatic_probabilities[-1, 0]
    print(f"Completed calculation for time_option: {time_option}, final_J_V_ratio: {final_J_V_ratio}, probability: {probability:.4f}")
    return (time_option, final_J_V_ratio, probability)

# Run the calculations in parallel
with ThreadPoolExecutor() as executor:
    futures = []
    for time_option in time_options:
        for final_J_V_ratio in final_J_V_ratios:
            futures.append(executor.submit(calculate_probability, time_option, final_J_V_ratio))

    for future in futures:
        time_option, final_J_V_ratio, probability = future.result()
        idx1 = np.where(time_options == time_option)[0][0]
        idx2 = np.where(final_J_V_ratios == final_J_V_ratio)[0][0]
        probability_grid[idx2, idx1] = probability

# Plotting
plt.pcolormesh(time_options_grid, final_J_V_ratios_grid, probability_grid, shading="auto", cmap="plasma", vmin=0, vmax=1)
plt.colorbar(label="Ground State Probability")
plt.xlabel("$T/4$")
plt.ylabel("$J/|V|$")
plt.title("Lower Bound Probability Grid")
plt.xlim(0, 1.6)
plt.show()


: 

: 

In [ ]:
# Exaggerate the time axis by a factor of 4
exaggerated_time_options_grid = time_options_grid * 4

# Plotting
plt.pcolormesh(exaggerated_time_options_grid, final_J_V_ratios_grid, probability_grid, shading="auto", cmap="plasma", vmin=0, vmax=1)
plt.colorbar(label="Ground State Probability")
plt.xlabel("$4T/4$")
plt.ylabel("$J/|V|$")
plt.title("Lower Bound Probability Grid")
plt.xlim(0, 4 * 1.6)  # Adjust xlim to match the exaggerated scale
plt.show()
